In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
from py2neo import Graph
graph = Graph()

In [4]:
folder = '/home/manikaran/M.tech/Minor project/data/CSR_NEW_FILES'

In [5]:
districtsDf = pd.read_csv('/home/manikaran/M.tech/Minor project/data/CSR_Districts_AspirantDistricts.csv')
districtsDf.head()

,Unnamed: 0,State,District,Aspirant_District
0,0,Andaman and Nicobar,Nicobar,0
1,1,Andaman and Nicobar,North Middle Andaman,0
2,2,Andaman and Nicobar,South Andaman,0
3,3,Andhra Pradesh,Anantapur,0
4,4,Andhra Pradesh,Chittoor,0


In [6]:
districtToConstDf = pd.read_csv(folder + '/districtsToConstituency.csv')

In [7]:
districtToConstDf.head()

,Unnamed: 0,district,state,constituency
0,0,Nicobar,Andaman and Nicobar,['ANDAMAN and NICOBAR']
1,1,North Middle Andaman,Andaman and Nicobar,['ANDAMAN and NICOBAR']
2,2,South Andaman,Andaman and Nicobar,['ANDAMAN and NICOBAR']
3,3,Anantapur,Andhra Pradesh,"['ANANTAPUR', 'HINDUPUR']"
4,4,Chittoor,Andhra Pradesh,"['CHITTOOR', 'RAJAMPET', 'TIRUPATI']"


In [8]:
data = []
fy = '2014-15'
for i in range(districtsDf.shape[0]):
    state = districtsDf.iloc[i]['State']
    district = districtsDf.iloc[i]['District']
    path = '{}/{}/{}/{}.csv'.format(folder, fy, state, district)
    tempDf = pd.read_csv(path)
    for c in range(tempDf.shape[0]):
        data.append([tempDf.iloc[c]['company'], state, district, tempDf.iloc[c]['amount']])

In [9]:
df = pd.DataFrame(data, columns=['company', 'state', 'district', 'amount'])

In [10]:
df.head()

,company,state,district,amount
0,Sundaram -Clayton Limited,Andhra Pradesh,Anantapur,"174,300,000.00"
1,S.G.P.Exim Private Limited,Andhra Pradesh,Anantapur,"10,200,000.00"
2,Gerdau Steel India Private Limited,Andhra Pradesh,Anantapur,"3,700,000.00"
3,The New India Assurance Company Limited,Andhra Pradesh,Anantapur,"2,500,000.00"
4,Om Telecom Logistics Private Limited,Andhra Pradesh,Anantapur,"1,400,000.00"


In [11]:
df.shape

(3382, 4)

In [12]:
df = df.merge(districtToConstDf, how='inner', left_on=['state', 'district'], right_on=['state', 'district'])

In [13]:
df.shape

(3382, 6)

In [14]:
pd.isna(df['company']).any()

False

In [15]:
df['amount'] = df['amount'].apply(lambda x: float(str(x).replace(',', '')))

In [16]:
compDf = pd.read_csv('{}/all_companies_with_CIN2014.csv'.format(folder), encoding='utf-7')

In [17]:
compDf.head()

,Unnamed: 0,Unnamed: 0.1,company,CIN
0,0,0,SUNDARAM -CLAYTON LIMITED,L35999TN1962PLC004792
1,1,1,S.G.P.EXIM PRIVATE LIMITED,U52321TN1992PTC022428
2,2,2,GERDAU STEEL INDIA PRIVATE LIMITED,U27109AP1993PTC015499
3,3,3,THE NEW INDIA ASSURANCE COMPANY LIMITED,U99999MH1919GOI000526
4,4,4,OM TELECOM LOGISTICS PRIVATE LIMITED,U63023DL2001PTC112366


In [18]:
df['company'] = df['company'].apply(lambda x: x.strip().upper())

In [19]:
df = df.merge(compDf[['company', 'CIN']], how='inner', on='company')

In [20]:
df.shape

(3382, 7)

In [21]:
df.head()

,company,state,district,amount,Unnamed: 0,constituency,CIN
0,SUNDARAM -CLAYTON LIMITED,Andhra Pradesh,Anantapur,174300000.0,3,"['ANANTAPUR', 'HINDUPUR']",L35999TN1962PLC004792
1,S.G.P.EXIM PRIVATE LIMITED,Andhra Pradesh,Anantapur,10200000.0,3,"['ANANTAPUR', 'HINDUPUR']",U52321TN1992PTC022428
2,S.G.P.EXIM PRIVATE LIMITED,Tamil Nadu,Chennai,1500000.0,461,"['CHENNAI CENTRAL', 'CHENNAI NORTH', 'CHENNAI ...",U52321TN1992PTC022428
3,GERDAU STEEL INDIA PRIVATE LIMITED,Andhra Pradesh,Anantapur,3700000.0,3,"['ANANTAPUR', 'HINDUPUR']",U27109AP1993PTC015499
4,THE NEW INDIA ASSURANCE COMPANY LIMITED,Andhra Pradesh,Anantapur,2500000.0,3,"['ANANTAPUR', 'HINDUPUR']",U99999MH1919GOI000526


In [22]:
lokSabhaDf = pd.read_csv(folder + '/LokSabha2014.csv')

In [23]:
lokSabhaDf.head()

,Unnamed: 0,Constituency,No,Type,State,Winning Candidate,Party,Electors,Votes,Turnout,Margin,Margin %,PartyShort
0,0,ADILABAD,1,ST,Telangana,Godam Nagesh,Telangana Rashtra Samithi,1386233,1045839,75.4 %,171290,16.4%,TRS
1,1,AGRA,18,SC,Uttar Pradesh,Dr. Ram Shankar Katheria,Bharatiya Janta Party,1814739,1070400,59.0 %,300263,28.1%,BJP
2,2,AHMEDNAGAR,37,GEN,Maharashtra,Gandhi Dilipkumar Mansukhlal,Bharatiya Janta Party,1699408,1062318,62.5 %,209122,19.7%,BJP
3,3,AHMEDABAD EAST,7,GEN,Gujarat,Paresh Rawal,Bharatiya Janta Party,1601832,985525,61.5 %,326633,33.1%,BJP
4,4,AHMEDABAD WEST,8,SC,Gujarat,Dr. Kirit P Solanki,Bharatiya Janta Party,1534400,964609,62.9 %,320311,33.2%,BJP


In [24]:
mps = []
party = []
voteMargin = []
type = []
for i in range(df.shape[0]):
    state = df.iloc[i]['state']
    constituency = eval(df.iloc[i]['constituency'].strip())
    mp_temp = []
    party_temp = []
    type_temp = []
    vote_margin_temp = []
    for c in constituency:
        tempDf = lokSabhaDf[(lokSabhaDf['Constituency'] == c.strip().upper()) & (lokSabhaDf['State'] == state)]
        mp_temp.append(tempDf.iloc[0]['Winning Candidate'])
        party_temp.append(tempDf.iloc[0]['PartyShort'])
        vote_margin_temp.append(tempDf.iloc[0]['Margin %'])
        type_temp.append(tempDf.iloc[0]['Type'])
    mps.append(mp_temp)
    party.append(party_temp)
    voteMargin.append(vote_margin_temp)
    type.append(type_temp)

In [25]:
df['MP'] = mps
df['party'] = party
df['voteMargin %'] = voteMargin
df['type'] = type

In [26]:
notMatchedDf= df[pd.isna(df['CIN'])]
notMatchedDf.shape

(0, 11)

In [27]:
from py2neo import Graph

In [28]:
graph = Graph()

In [29]:
# finding companies not in graph db
comp_present_query = 'match (c:company) where c.cin = "{}" return c'
present = [False]*df.shape[0]
for i in range(df.shape[0]):
    res = graph.data(comp_present_query.format(df.iloc[i]['CIN'].strip()))
    if len(res) > 0:
        present[i] = True

In [30]:
notPresent = df.shape[0] - sum(present)

In [31]:
notPresent

1107

In [32]:
df['present_in_graph_db'] = present

In [33]:
df.shape

(3382, 12)

In [58]:
df.head()

,company,state,district,amount,Unnamed: 0,constituency,CIN,MP,party,voteMargin %,type,present_in_graph_db
0,SUNDARAM -CLAYTON LIMITED,Andhra Pradesh,Anantapur,174300000.0,3,"['ANANTAPUR', 'HINDUPUR']",L35999TN1962PLC004792,"[J.C. Divakar Reddi, Kristappa Nimmala]","[TDP, TDP]","[5.1%, 8.3%]","[GEN, GEN]",True
1,S.G.P.EXIM PRIVATE LIMITED,Andhra Pradesh,Anantapur,10200000.0,3,"['ANANTAPUR', 'HINDUPUR']",U52321TN1992PTC022428,"[J.C. Divakar Reddi, Kristappa Nimmala]","[TDP, TDP]","[5.1%, 8.3%]","[GEN, GEN]",False
2,S.G.P.EXIM PRIVATE LIMITED,Tamil Nadu,Chennai,1500000.0,461,"['CHENNAI CENTRAL', 'CHENNAI NORTH', 'CHENNAI ...",U52321TN1992PTC022428,"[S.R. Vijayakumar, Venkatesh Babu .T.G, Dr. J....","[AIADMK, AIADMK, AIADMK]","[5.6%, 11.0%, 12.6%]","[GEN, GEN, GEN]",False
3,GERDAU STEEL INDIA PRIVATE LIMITED,Andhra Pradesh,Anantapur,3700000.0,3,"['ANANTAPUR', 'HINDUPUR']",U27109AP1993PTC015499,"[J.C. Divakar Reddi, Kristappa Nimmala]","[TDP, TDP]","[5.1%, 8.3%]","[GEN, GEN]",False
4,THE NEW INDIA ASSURANCE COMPANY LIMITED,Andhra Pradesh,Anantapur,2500000.0,3,"['ANANTAPUR', 'HINDUPUR']",U99999MH1919GOI000526,"[J.C. Divakar Reddi, Kristappa Nimmala]","[TDP, TDP]","[5.1%, 8.3%]","[GEN, GEN]",True


In [59]:
def Or(li):
    res = False
    for x in li:
        res = res or x
    return res
def concat(li):
    res = []
    for x in li:
        res += x
    return res
groupedDf = (df.groupby(['CIN', 'company']).agg({'amount':sum, 'state':list, 'district':list, 'present_in_graph_db':Or, 'constituency':list, 'MP':list, 'party':list, 'voteMargin %':list})).reset_index()

In [60]:
avg = groupedDf[groupedDf['amount'] != 0]['amount'].mean()
sd = groupedDf[groupedDf['amount'] != 0]['amount'].std()
thresh = avg + sd
avg, sd, thresh

(9677123.22042219, 69379466.55984466, 79056589.78026685)

In [61]:
groupedDf[(groupedDf['amount'] > thresh) & (groupedDf['present_in_graph_db'] == False)]

,CIN,company,amount,state,district,present_in_graph_db,constituency,MP,party,voteMargin %
785,U24232KA1973PLC002383,MEDREICH LIMITED,108000000.0,[Karnataka],[Bangalore Rural],False,"[['BANGALORE RURAL', 'CHIKKBALLAPUR']]","[[D K Suresh, M Veerappa Moily]]","[[INC, INC]]","[[15.9%, 0.8%]]"
1571,U65929MH2003PTC141455,PRESCIENT SECURITIES PRIVATE LIMITED,100000000.0,[Maharashtra],[Pune],False,"[['BARAMATI', 'MAVAL', 'PUNE', 'SHIRUR']]","[[Supriya Sule, Appa Alias Shrirang Chandu Bar...","[[NCP, SS, BJP, SS]]","[[6.5%, 13.4%, 31.8%, 27.7%]]"
1584,U65990MH1993PTC073769,GAGANDEEP CREDIT CAPITAL PRIVATE LIMITED,250000000.0,[Maharashtra],[Pune],False,"[['BARAMATI', 'MAVAL', 'PUNE', 'SHIRUR']]","[[Supriya Sule, Appa Alias Shrirang Chandu Bar...","[[NCP, SS, BJP, SS]]","[[6.5%, 13.4%, 31.8%, 27.7%]]"


In [62]:
groupedDf.sort_values(by='amount', inplace=True, ascending=False)

In [63]:
top10percentDf = groupedDf.iloc[0:df.shape[0]//10]
(top10percentDf[top10percentDf['present_in_graph_db'] == False]).shape

(49, 10)

In [64]:
top10percentDf[top10percentDf['present_in_graph_db'] == False]

,CIN,company,amount,state,district,present_in_graph_db,constituency,MP,party,voteMargin %
1584,U65990MH1993PTC073769,GAGANDEEP CREDIT CAPITAL PRIVATE LIMITED,250000000.0,[Maharashtra],[Pune],False,"[['BARAMATI', 'MAVAL', 'PUNE', 'SHIRUR']]","[[Supriya Sule, Appa Alias Shrirang Chandu Bar...","[[NCP, SS, BJP, SS]]","[[6.5%, 13.4%, 31.8%, 27.7%]]"
785,U24232KA1973PLC002383,MEDREICH LIMITED,108000000.0,[Karnataka],[Bangalore Rural],False,"[['BANGALORE RURAL', 'CHIKKBALLAPUR']]","[[D K Suresh, M Veerappa Moily]]","[[INC, INC]]","[[15.9%, 0.8%]]"
1571,U65929MH2003PTC141455,PRESCIENT SECURITIES PRIVATE LIMITED,100000000.0,[Maharashtra],[Pune],False,"[['BARAMATI', 'MAVAL', 'PUNE', 'SHIRUR']]","[[Supriya Sule, Appa Alias Shrirang Chandu Bar...","[[NCP, SS, BJP, SS]]","[[6.5%, 13.4%, 31.8%, 27.7%]]"
467,U01100TG1986PTC112216,AJEET SEEDS PRIVATE LIMITED,40200000.0,[Maharashtra],[Aurangabad],False,"[['JALNA', 'AURANGABAD']]","[[Danve Raosaheb Dadarao, Chandrakant Bhaurao ...","[[BJP, SS]]","[[19.4%, 16.5%]]"
1720,U70101DL2004PTC125163,IREO PRIVATE LIMITED,36900000.0,[Haryana],[Gurugram],False,[['GURGAON']],[[Inderjit Singh Rao]],[[BJP]],[[20.8%]]
1193,U40102TG2001PTC035962,ELGEN (INDIA) PRIVATE LIMITED,35100000.0,[Telangana],[Karimnagar],False,"[['KARIMNAGAR', 'NIZAMABAD', 'PEDDAPALLE']]","[[Vinod Kumar Boinapally, Kalvakuntla Kavitha,...","[[TRS, TRS, TRS]]","[[18.2%, 16.2%, 28.5%]]"
764,U24230TN2007PTC064220,PALEPU PHARMA PRIVATE LIMITED,31200000.0,"[Tamil Nadu, Telangana]","[Chennai, Hyderabad]",False,"[['CHENNAI CENTRAL', 'CHENNAI NORTH', 'CHENNAI...","[[S.R. Vijayakumar, Venkatesh Babu .T.G, Dr. J...","[[AIADMK, AIADMK, AIADMK], [AIMM, TDP, BJP]]","[[5.6%, 11.0%, 12.6%], [20.8%, 1.8%, 25.4%]]"
1874,U72400KA2003PTC032606,GOLDMAN SACHS SERVICES PRIVATE LIMITED,30500000.0,[Karnataka],[Bangalore Urban],False,"[['BANGALORE CENTRAL', 'BANGALORE NORTH', 'BAN...","[[P.C. Mohan, D.V Sadananda Gowda, D K Suresh,...","[[BJP, BJP, INC, BJP]]","[[12.8%, 16.9%, 15.9%, 20.5%]]"
1557,U65923MH1997PTC106632,BENGAL FINANCE AND INVESTMENT PRIVATE LIMITED,30400000.0,"[Assam, Delhi, Gujarat, Gujarat, Gujarat, Madh...","[Bongaigaon, New Delhi, Ahmedabad, Rajkot, Val...",False,"[['BARPETA', 'KOKRAJHAR'], ['EAST DELHI', 'NEW...","[[Siraj Uddin Ajmal, Naba Kumar Sarania (HIRA)...","[[AIUDF, Ind], [BJP, BJP], [BJP, BJP, BJP, BJP...","[[3.5%, 29.1%], [15.9%, 16.8%], [33.1%, 33.2%,..."
605,U16003MH1991PTC061695,BORSAD TOBACCO COMPANY PRIVATE LIMITED,27200000.0,[Maharashtra],[Nandurbar],False,[['NANDURBAR']],[[Dr.Gavit Heena Vaijaykumar]],[[BJP]],[[9.6%]]


In [65]:
thresh = 7e06
aboveThreshDf = groupedDf[groupedDf['amount'] >= thresh]
aboveThreshDf.shape

(340, 10)

In [66]:
notInGraphDbDf = (aboveThreshDf[aboveThreshDf['present_in_graph_db'] == False])
notInGraphDbDf.shape

(50, 10)

In [67]:
notInGraphDbDf.head()

,CIN,company,amount,state,district,present_in_graph_db,constituency,MP,party,voteMargin %
1584,U65990MH1993PTC073769,GAGANDEEP CREDIT CAPITAL PRIVATE LIMITED,250000000.0,[Maharashtra],[Pune],False,"[['BARAMATI', 'MAVAL', 'PUNE', 'SHIRUR']]","[[Supriya Sule, Appa Alias Shrirang Chandu Bar...","[[NCP, SS, BJP, SS]]","[[6.5%, 13.4%, 31.8%, 27.7%]]"
785,U24232KA1973PLC002383,MEDREICH LIMITED,108000000.0,[Karnataka],[Bangalore Rural],False,"[['BANGALORE RURAL', 'CHIKKBALLAPUR']]","[[D K Suresh, M Veerappa Moily]]","[[INC, INC]]","[[15.9%, 0.8%]]"
1571,U65929MH2003PTC141455,PRESCIENT SECURITIES PRIVATE LIMITED,100000000.0,[Maharashtra],[Pune],False,"[['BARAMATI', 'MAVAL', 'PUNE', 'SHIRUR']]","[[Supriya Sule, Appa Alias Shrirang Chandu Bar...","[[NCP, SS, BJP, SS]]","[[6.5%, 13.4%, 31.8%, 27.7%]]"
467,U01100TG1986PTC112216,AJEET SEEDS PRIVATE LIMITED,40200000.0,[Maharashtra],[Aurangabad],False,"[['JALNA', 'AURANGABAD']]","[[Danve Raosaheb Dadarao, Chandrakant Bhaurao ...","[[BJP, SS]]","[[19.4%, 16.5%]]"
1720,U70101DL2004PTC125163,IREO PRIVATE LIMITED,36900000.0,[Haryana],[Gurugram],False,[['GURGAON']],[[Inderjit Singh Rao]],[[BJP]],[[20.8%]]


In [68]:
notInGraphDbDf['amount (in Lakhs)'] = notInGraphDbDf.loc[:,'amount']/100000

/home/manikaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [113]:
ministersDf = pd.read_csv('{}/ministers2014-15Final.csv'.format(folder))
ministersDf['isMPMinister'] = [1]*ministersDf.shape[0]
ministersDf['LokSabhaConstituency'] = ministersDf['LokSabhaConstituency'].apply(str.upper)
ministersDf.head()

,Unnamed: 0,ministers,ministry,minCode,LokSabhaConstituency,isMPMinister
0,0,Col Rajyavardhan Singh Rathore (Retd.) AVSM,['Information and Broadcasting'],[19.0],JAIPUR RURAL,1
1,1,Dr. Harsh Vardhan,"['Science and Technology', 'Earth Sciences']","[6.0, 23.0]",CHANDNI CHOWK,1
2,2,Dr. Jitendra Singh,['Development of North Eastern Region (Indepen...,"[20.0, 39.0, 8.0]",UDHAMPUR,1
3,3,Dr. Mahesh Sharma,"['Tourism (Independent Charge)', 'Culture (Ind...","[14.0, 2.0, 36.0]",GAUTAM BUDHHA NAGAR,1
4,4,Dr. Sanjeev Kumar Balyan,['Agriculture'],[37.0],MUZAFFARNAGAR,1


In [114]:
ministry = []
for i in range(notInGraphDbDf.shape[0]):
    temp = []
    for c in eval(notInGraphDbDf.iloc[i]['constituency'][0]):
        tempDf = ministersDf[ministersDf['LokSabhaConstituency'] == c]
        if tempDf.shape[0] != 0:
            temp.append(eval(tempDf.iloc[0]['ministry']))
        else:
            temp.append('')
    ministry.append(temp)
notInGraphDbDf['ministry'] = ministry

/home/manikaran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [115]:
ministry

[['', '', '', ''],
 ['', ''],
 ['', '', '', ''],
 ['', ''],
 [['Planning (Independent Charge)']],
 ['', '', ''],
 ['', '', ''],
 ['', ['Law and Justice'], '', ['Chemicals and Fertilizers']],
 ['', ''],
 [''],
 ['', '', '', ''],
 ['', '', ''],
 [''],
 [['Social Justice and Empowerment']],
 ['', ''],
 ['', ''],
 [''],
 [''],
 ['', ''],
 [''],
 ['', '', ''],
 ['', ''],
 ['', ''],
 [''],
 ['', ''],
 [['Planning (Independent Charge)']],
 ['', ''],
 ['', ''],
 ['', ['Law and Justice'], '', ['Chemicals and Fertilizers']],
 ['', ''],
 ['', '', '', ''],
 ['', '', ''],
 ['', ''],
 ['', '', ''],
 ['', '', ''],
 ['', '', '', ''],
 ['', '', '', ''],
 [['Planning (Independent Charge)']],
 ['', ''],
 ['', '', ''],
 [['Planning (Independent Charge)']],
 ['', '', ['Labour and Employment (Independent Charge)']],
 ['', '', ['Labour and Employment (Independent Charge)']],
 ['', ''],
 ['', ''],
 ['', '', ''],
 ['', ''],
 ['', ''],
 ['', '', ''],
 ['', '']]

In [69]:
director_crawl_url = 'https://www.zaubacorp.com/company-directors/{}/{}'

In [45]:
data = []
for i in range(notInGraphDbDf.shape[0]):
    response = requests.get(director_crawl_url.format(notInGraphDbDf.iloc[i]['company'].replace(' ', '-'), notInGraphDbDf.iloc[i]['CIN']))
    if response.ok:
        html_page = response.text
        soup = BeautifulSoup(html_page, 'html.parser')
        table = soup.find('table', attrs={'class':'table-striped'})
        header = table.findAll('th')
        if header[0].text.strip() == 'CIN':
            print(director_crawl_url.format(notInGraphDbDf.iloc[i]['company'].replace(' ', '-'), notInGraphDbDf.iloc[i]['CIN']), 'not found')
            continue
        rows = table.findAll('tr')
        for r in rows:
            cols = r.findAll('td')
            if len(cols) == 0:
                continue
            temp= [notInGraphDbDf.iloc[i]['CIN']]
            for c in cols:
                temp.append(c.text.strip())
            data.append(temp)

In [46]:
cins = []

In [48]:
directorsDf = pd.DataFrame(data)

In [49]:
directorsDf.isna().any()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [50]:
directorsDf.to_csv('newCompDirectors.csv')

In [51]:
directorsDf.head()

,0,1,2,3,4
0,U65990MH1993PTC073769,00619050,JAYESH KANTILAL SHAH,Director,06 September 1993
1,U65990MH1993PTC073769,00619040,ANUJ ANANTRAI SHETH,Director,12 January 2012
2,U65990MH1993PTC073769,05167220,AAGAM ANUJKUMAR SHETH,Director,12 January 2012
3,U24232KA1973PLC002383,03486485,NARASIMHAN SESHADRI,Director,31 March 2015
4,U24232KA1973PLC002383,08166228,KENSHI MURASE,Director,02 July 2018


In [52]:
directorsDf.shape[0]

207

In [53]:
directorsDf.columns = ['CIN', 'DIN', 'name', 'designation', 'dateOfAppointment']

In [56]:
groupedDirDf = directorsDf.groupby('CIN').agg({'DIN':list, 'name':list, 'designation':list, 'dateOfAppointment':list}).reset_index()

In [118]:
newCompanyDf = notInGraphDbDf.merge(groupedDirDf[['CIN', 'name']], how='inner', on='CIN')

In [119]:
newCompanyDf.rename({'name':'directorName'}, inplace=True, axis=1)

In [120]:
newCompanyDf.columns

Index(['CIN', 'company', 'amount', 'state', 'district', 'present_in_graph_db',
       'constituency', 'MP', 'party', 'voteMargin %', 'amount (in Lakhs)',
       'ministry', 'directorName'],
      dtype='object')

In [121]:
newCompanyDf[['CIN', 'company', 'directorName', 'amount (in Lakhs)', 'state', 'district', 'constituency', 'MP', 'ministry', 'party', 'voteMargin %']].reset_index(drop=True).to_csv('new_companies.csv')

In [122]:
newCompanyDf[['CIN', 'company', 'directorName', 'amount (in Lakhs)', 'state', 'district', 'constituency', 'MP', 'ministry', 'party', 'voteMargin %']].reset_index(drop=True).to_excel('new_companies.xlsx')

In [103]:
dinIsPresentQuery = 'match (d:businessperson) where d.din = "{}" return count(d) as cnt'
present = []
for din in directorsDf['DIN']:
    present.append(graph.data(dinIsPresentQuery)[0]['cnt'])
print(sum(present), 'are in graph db out of', len(present))

0 are in graph db out of 191


In [104]:
bod_dob_file = '/home/manikaran/M.tech/Minor project/data/dob_data_b.csv'

In [105]:
directorDobDf = pd.read_csv(bod_dob_file)

In [106]:
directorDobDf.head()

,Unnamed: 0,din,name,dob
0,0,1,RATAN NAVAL TATA,28/12/1937
1,1,2,RAMADORAI SUBRAMANIAN,06/10/1944
2,2,555,YUDHVIR SINGH MALIK,19/03/1959
3,3,1016,AJAY BHAGAT,06/08/1971
4,4,1033,RAMANATHAN NARAYANAN,14/06/1954


In [107]:
directorDobDf['DIN'] = directorDobDf['din'].apply(lambda x: '{:0>8}'.format(x))

In [112]:
directorsDf.shape

(191, 5)

In [108]:
tempDf = (directorsDf.merge(directorDobDf[['DIN', 'name', 'dob']], how='inner', on='DIN'))

In [111]:
tempDf.shape

(57, 7)

In [110]:
# not required

import pymysql

cnx = pymysql.Connect(db='mca', user='root', passwd='mkarank@001')

cursor = cnx.cursor()

query = 'select cin from company where upper(name)="{}"'

cin = []
for i in range(notMatchedDf.shape[0]):
    cursor.execute(query.format(notMatchedDf.iloc[i]['company']))
    res = cursor.fetchone()
    if res is None:
        cin.append(None)
        print(notMatchedDf.iloc[i]['company'])
    else:
        cin.append(res[0])

cursor.close()

cin